# Prompt sail proxy for OpenAI with LangChain

In [3]:
from langchain.llms import OpenAI


import os
from dotenv import load_dotenv
load_dotenv()

print(os.getenv("OPENAI_API_KEY"))

NameError: name 'os' is not defined

Visit http://project1.promptsail.local

In [2]:
llm = OpenAI(
    model_name="text-davinci-003",
    openai_api_base="http://project1.promptsail.local:8000",
)
llm("Explaining the meaning of life in one sentence.")

ValidationError: 1 validation error for OpenAI
__root__
  Did not find openai_api_key, please add an environment variable `OPENAI_API_KEY` which contains it, or pass  `openai_api_key` as a named parameter. (type=value_error)